In [1]:
# Slighlty updated for Keras 2.0 APIs
# - by Kyeong Soo (Joseph) Kim

import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
path_train = "./UJIndoorLoc/trainingData2.csv"
path_validation = "./UJIndoorLoc/validationData2.csv"

In [3]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.iloc[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths).astype(float), axis=1) # convert integer to float and scale jointly (axis=1)

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

In [4]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [5]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [6]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]).astype(float))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [7]:
epochs = 20
batch_size = 10
input_size = 520
num_classes = 13
nums_hidden_nodes = [16, 32, 64, 128, 256, 512]
# nums_hidden_nodes = [16]
# dropout_rates = np.arange(11)*0.05 # 0.0, 0.05,...,0.5
dropout_rates = [0.0, 0.2, 0.5] # for test
# dropout_rates = [0.5] # for test
losses = np.zeros((len(nums_hidden_nodes), len(dropout_rates)))
accuracies = np.zeros_like(losses)

In [8]:
def classifier(num_hidden_nodes, dropout_rate):
    model = Sequential()
    model.add(Dense(num_hidden_nodes, input_dim=input_size, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(256, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(128, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(64, activation='tanh', use_bias=True))
#     model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax', use_bias=True))
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [9]:
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        c = classifier(nums_hidden_nodes[nh], dropout_rates[dr])
        startTime = timer()
        c.fit(train_X, train_y, validation_data=(val_X, val_y), batch_size=batch_size, epochs=epochs)
        elapsedTime = timer() - startTime
        print("Model trained with nh=%d & dr=%.2f in %e s.\n" % (nums_hidden_nodes[nh], dropout_rates[dr], elapsedTime))
        loss, acc = c.evaluate(test_AP_features, test_labels)
        losses[nh, dr] = loss
        accuracies[nh, dr] = acc

Train on 13986 samples, validate on 5944 samples
Epoch 1/20
13986/13986 [==============================] - 2s - loss: 0.3903 - acc: 0.9248 - val_loss: 0.0823 - val_acc: 0.9887
Epoch 2/20
13986/13986 [==============================] - 2s - loss: 0.0537 - acc: 0.9904 - val_loss: 0.0449 - val_acc: 0.9881
Epoch 3/20
13986/13986 [==============================] - 2s - loss: 0.0299 - acc: 0.9931 - val_loss: 0.0292 - val_acc: 0.99260.996 - ETA: 1s - 
Epoch 4/20
13986/13986 [==============================] - 2s - loss: 0.0200 - acc: 0.9942 - val_loss: 0.0278 - val_acc: 0.9919
Epoch 5/20
13986/13986 [==============================] - 2s - loss: 0.0146 - acc: 0.9954 - val_loss: 0.0218 - val_acc: 0.9936
Epoch 6/20
13986/13986 [==============================] - 2s - loss: 0.0128 - acc: 0.9961 - val_loss: 0.0266 - val_acc: 0.9916
Epoch 7/20
13986/13986 [==============================] - 2s - loss: 0.0122 - acc: 0.9953 - val_loss: 0.0289 - val_acc: 0.9902
Epoch 8/20
13986/13986 [====================

13986/13986 [==============================] - 3s - loss: 0.2304 - acc: 0.9490 - val_loss: 0.0530 - val_acc: 0.9897
Epoch 2/20
13986/13986 [==============================] - 3s - loss: 0.0357 - acc: 0.9913 - val_loss: 0.0310 - val_acc: 0.9923
Epoch 3/20
13986/13986 [==============================] - 3s - loss: 0.0225 - acc: 0.9935 - val_loss: 0.0305 - val_acc: 0.9902
Epoch 4/20
13986/13986 [==============================] - 3s - loss: 0.0147 - acc: 0.9947 - val_loss: 0.0293 - val_acc: 0.9882
Epoch 5/20
13986/13986 [==============================] - 3s - loss: 0.0126 - acc: 0.9954 - val_loss: 0.0195 - val_acc: 0.9929
Epoch 6/20
13986/13986 [==============================] - 3s - loss: 0.0109 - acc: 0.9954 - val_loss: 0.0199 - val_acc: 0.9938
Epoch 7/20
13986/13986 [==============================] - 3s - loss: 0.0089 - acc: 0.9963 - val_loss: 0.0187 - val_acc: 0.9946
Epoch 8/20
13986/13986 [==============================] - 3s - loss: 0.0071 - acc: 0.9964 - val_loss: 0.0293 - val_acc: 0.

13986/13986 [==============================] - 3s - loss: 0.1693 - acc: 0.9585 - val_loss: 0.0438 - val_acc: 0.9902
Epoch 2/20
13986/13986 [==============================] - 2s - loss: 0.0277 - acc: 0.9934 - val_loss: 0.0290 - val_acc: 0.9918
Epoch 3/20
13986/13986 [==============================] - 2s - loss: 0.0173 - acc: 0.9940 - val_loss: 0.0234 - val_acc: 0.9924
Epoch 4/20
13986/13986 [==============================] - 2s - loss: 0.0123 - acc: 0.9959 - val_loss: 0.0199 - val_acc: 0.9928
Epoch 5/20
13986/13986 [==============================] - 2s - loss: 0.0116 - acc: 0.9954 - val_loss: 0.0221 - val_acc: 0.9924
Epoch 6/20
13986/13986 [==============================] - 2s - loss: 0.0094 - acc: 0.9962 - val_loss: 0.0142 - val_acc: 0.9955
Epoch 7/20
13986/13986 [==============================] - 2s - loss: 0.0069 - acc: 0.9969 - val_loss: 0.0145 - val_acc: 0.9956
Epoch 8/20
13986/13986 [==============================] - 3s - loss: 0.0069 - acc: 0.9968 - val_loss: 0.0173 - val_acc: 0.

13986/13986 [==============================] - 4s - loss: 0.1306 - acc: 0.9654 - val_loss: 0.0282 - val_acc: 0.9946
Epoch 2/20
13986/13986 [==============================] - 4s - loss: 0.0273 - acc: 0.9910 - val_loss: 0.0194 - val_acc: 0.9936
Epoch 3/20
13986/13986 [==============================] - 4s - loss: 0.0170 - acc: 0.9936 - val_loss: 0.0185 - val_acc: 0.9934
Epoch 4/20
13986/13986 [==============================] - 4s - loss: 0.0138 - acc: 0.9951 - val_loss: 0.0314 - val_acc: 0.9897
Epoch 5/20
13986/13986 [==============================] - 4s - loss: 0.0088 - acc: 0.9965 - val_loss: 0.0134 - val_acc: 0.9960
Epoch 6/20
13986/13986 [==============================] - 4s - loss: 0.0103 - acc: 0.9958 - val_loss: 0.0391 - val_acc: 0.9852
Epoch 7/20
13986/13986 [==============================] - 4s - loss: 0.0093 - acc: 0.9968 - val_loss: 0.0138 - val_acc: 0.9950
Epoch 8/20
13986/13986 [==============================] - 4s - loss: 0.0075 - acc: 0.9966 - val_loss: 0.0134 - val_acc: 0.

13986/13986 [==============================] - 5s - loss: 0.1062 - acc: 0.9689 - val_loss: 0.0485 - val_acc: 0.9844
Epoch 2/20
13986/13986 [==============================] - 5s - loss: 0.0296 - acc: 0.9906 - val_loss: 0.0282 - val_acc: 0.9909
Epoch 3/20
13986/13986 [==============================] - 5s - loss: 0.0221 - acc: 0.9920 - val_loss: 0.0752 - val_acc: 0.9807
Epoch 4/20
13986/13986 [==============================] - 5s - loss: 0.0166 - acc: 0.9932 - val_loss: 0.0191 - val_acc: 0.9936
Epoch 5/20
13986/13986 [==============================] - 5s - loss: 0.0143 - acc: 0.9948 - val_loss: 0.0224 - val_acc: 0.9936
Epoch 6/20
13986/13986 [==============================] - 5s - loss: 0.0118 - acc: 0.9959 - val_loss: 0.0192 - val_acc: 0.9943
Epoch 7/20
13986/13986 [==============================] - 5s - loss: 0.0105 - acc: 0.9959 - val_loss: 0.0203 - val_acc: 0.9941
Epoch 8/20
13986/13986 [==============================] - 5s - loss: 0.0090 - acc: 0.9967 - val_loss: 0.0168 - val_acc: 0.

13986/13986 [==============================] - 11s - loss: 0.1087 - acc: 0.9655 - val_loss: 0.0596 - val_acc: 0.9805
Epoch 2/20
13986/13986 [==============================] - 10s - loss: 0.0390 - acc: 0.9866 - val_loss: 0.0512 - val_acc: 0.9830
Epoch 3/20
13986/13986 [==============================] - 10s - loss: 0.0380 - acc: 0.9892 - val_loss: 0.0281 - val_acc: 0.9911
Epoch 4/20
13986/13986 [==============================] - 10s - loss: 0.0196 - acc: 0.9933 - val_loss: 0.0447 - val_acc: 0.9864
Epoch 5/20
13986/13986 [==============================] - 10s - loss: 0.0205 - acc: 0.9926 - val_loss: 0.0317 - val_acc: 0.9919
Epoch 6/20
13986/13986 [==============================] - 10s - loss: 0.0215 - acc: 0.9934 - val_loss: 0.0278 - val_acc: 0.9933
Epoch 7/20
13986/13986 [==============================] - 10s - loss: 0.0223 - acc: 0.9934 - val_loss: 0.0271 - val_acc: 0.9934
Epoch 8/20
13986/13986 [==============================] - 10s - loss: 0.0137 - acc: 0.9951 - val_loss: 0.0622 - val

In [10]:
for nh in range(len(nums_hidden_nodes)):
    for dr in range(len(dropout_rates)):
        print("nh=%d & dr=%.2f: loss=%e, accuracy=%e" % (nums_hidden_nodes[nh], dropout_rates[dr], losses[nh, dr], accuracies[nh, dr]))

nh=16 & dr=0.00: loss=1.234082e+00, accuracy=7.533753e-01

nh=16 & dr=0.20: loss=1.492540e+00, accuracy=7.353735e-01

nh=16 & dr=0.50: loss=1.369105e+00, accuracy=7.488749e-01

nh=32 & dr=0.00: loss=1.026828e+00, accuracy=8.001800e-01

nh=32 & dr=0.20: loss=1.256892e+00, accuracy=7.560756e-01

nh=32 & dr=0.50: loss=1.296779e+00, accuracy=7.659766e-01

nh=64 & dr=0.00: loss=1.033471e+00, accuracy=7.947795e-01

nh=64 & dr=0.20: loss=9.504665e-01, accuracy=8.235824e-01

nh=64 & dr=0.50: loss=1.162159e+00, accuracy=7.830783e-01

nh=128 & dr=0.00: loss=1.055724e+00, accuracy=8.001800e-01

nh=128 & dr=0.20: loss=1.023210e+00, accuracy=8.343834e-01

nh=128 & dr=0.50: loss=1.475645e+00, accuracy=7.389739e-01

nh=256 & dr=0.00: loss=1.222873e+00, accuracy=7.947795e-01

nh=256 & dr=0.20: loss=9.960540e-01, accuracy=8.280828e-01

nh=256 & dr=0.50: loss=1.053368e+00, accuracy=8.127813e-01

nh=512 & dr=0.00: loss=1.344286e+00, accuracy=8.100810e-01

nh=512 & dr=0.20: loss=1.255742e+00, accuracy=7.9